In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt

import geopandas as gpd
from osgeo import gdal, ogr, osr
from shapely.geometry import Point
from gdalconst import GA_ReadOnly

import os, gc
import datetime

# Read dataset in gdal

In [ ]:
dem_path = os.path.join('data', 'NS_DEM.tif')

In [ ]:
ds = gdal.Open(dem_path)

In [ ]:
geotransform = ds.GetGeoTransform()
geotransform

In [ ]:
print(f"Origin = ({geotransform[0]}, {geotransform[3]})")
print(f"Pixel Size = ({geotransform[1]}, {geotransform[5]})")

In [ ]:
print("Driver: {}/{}".format(ds.GetDriver().ShortName, ds.GetDriver().LongName))
print("Size is {} x {} x {}".format(ds.RasterXSize,
                                    ds.RasterYSize,
                                    ds.RasterCount))
print("Projection is {}".format(ds.GetProjection()))

In [ ]:
# 讀取特定band的資料
bands = ds.GetRasterBand(1)
print(f"Band Type = {gdal.GetDataTypeName(bands.DataType)}")

# 抓取特定band的最大最小值
min_value = bands.GetMinimum()
max_value = bands.GetMaximum()
if not min_value or not max_value:
    (min_value, max_value) = bands.ComputeRasterMinMax(True)
print("Min={:.3f}, Max={:.3f}".format(min_value, max_value))

# 抓取特定band的 no data value
no_data_value = bands.GetNoDataValue()
print("No data value is : {}".format(no_data_value))

## Get array in band

In [ ]:
arr = bands.ReadAsArray(xoff=0, yoff=0,)
arr = np.where(arr==no_data_value, np.nan, arr)
arr.shape

In [ ]:
plt.figure(figsize=(8, 8))
plt.imshow(arr)
plt.colorbar()

# Dem processing

In [ ]:
 def hillshade(array, azimuth, angle_altitude):

    # 取自: http://geoexamples.blogspot.com.br/2014/03/shaded-relief-images-using-gdal-python.html

    x, y = np.gradient(array)
    slope = np.pi/2. - np.arctan(np.sqrt(x*x + y*y))
    aspect = np.arctan2(-x, y)
    azimuth = azimuth*np.pi / 180.
    altitude = angle_altitude*np.pi / 180.


    shade = np.sin(altitude) * np.sin(slope)  + np.cos(altitude) * np.cos(slope)   * np.cos(azimuth - aspect)
    return 255*(shade + 1)/2

![i](https://upload.wikimedia.org/wikipedia/commons/f/f7/Azimuth-Altitude_schematic.svg)

In [ ]:
plt.figure(figsize=(8, 8))
plt.imshow(hillshade(arr, 30, 30))

# Saved tif

In [ ]:
dst_rst_path = os.path.join('output', 'hillshade.tif')
ref_cols = ds.RasterXSize
ref_rows = ds.RasterYSize
ref_bands = ds.RasterCount
G_type = ds.GetRasterBand(1).DataType

# 找出原本DEM的driver
driver = ds.GetDriver()

# 以原本DEM的相關資料創建新的 ds
out_ds = driver.Create(dst_rst_path, ref_cols, ref_rows, ref_bands, gdal.GDT_Float32)

# 設定output ds的相關空間資訊
out_ds.SetGeoTransform(ds.GetGeoTransform())
out_ds.SetProjection(ds.GetProjection())

# 從output ds的band並寫入numpy網格資料
out_band = out_ds.GetRasterBand(1)
out_band.SetNoDataValue(-9999.)

out_band.WriteArray(hillshade(arr, 30, 30))
out_band.FlushCache()
out_ds = None

In [ ]:
out_ds = gdal.Open('output/hillshade.tif')
h_arr = out_ds.GetRasterBand(1).ReadAsArray(xoff=0, yoff=0,)
plt.imshow(np.where(h_arr==-9999, np.nan, h_arr))
out_ds = None

# Demo

In [ ]:
satellite_list = ['20190327']
use_bands = ['B04', 'B08']

In [ ]:
satellite_imgs = {
    d:{i.split("_")[-1][:3]:os.path.join("data", d, i) 
       for i in  os.listdir(os.path.join("data", d)) if i.split(".")[-1] == 'tif' and i.split("_")[-1][:3] in use_bands} 
    for d in satellite_list
}

In [ ]:
satellite_imgs

In [ ]:
class RasterExtractor():
    '''
    Read and parse raster files
    raster_path(str): the path of the raster file
        <input>
    '''
    def __init__(self, raster_path):
        self.raster = gdal.Open(raster_path)
        self.geotransform = self.raster.GetGeoTransform()
        self.band = self.raster.GetRasterBand(1)
    
    def get_info(self):
        '''
        Get the information of the raster file. Including Driver, 
        raster size, Projection, Origin, the Min and Max values, No data value...etc
        '''
        print("Driver: {}/{}".format(self.raster.GetDriver().ShortName,
                                     self.raster.GetDriver().LongName))
        print("Size is {} x {} x {}".format(self.raster.RasterXSize,
                                            self.raster.RasterYSize,
                                            self.raster.RasterCount))
        print("Projection is {}".format(self.raster.GetProjection()))

        if self.geotransform:
            print(f"Origin = ({self.geotransform[0]}, {self.geotransform[3]})")
            print(f"Pixel Size = ({self.geotransform[1]}, {self.geotransform[5]})")

        print(f"Band Type={gdal.GetDataTypeName(self.band.DataType)}")
        min_value = self.band.GetMinimum()
        max_value = self.band.GetMaximum()
        if not min_value or not max_value:
            (min_value, max_value) = self.band.ComputeRasterMinMax(True)
        print("Min={:.3f}, Max={:.3f}".format(min_value, max_value))
        if self.band.GetOverviewCount() > 0:
            print("Band has {} overviews".format(self.band.GetOverviewCount()))
        if self.band.GetRasterColorTable():
            print("Band has a color table with {} entries".format(self.band.GetRasterColorTable().GetCount()))
        no_data_value = self.band.GetNoDataValue()
        print("No data value is : {}".format(no_data_value))
    
    def get_array(self):
        ''''''
        no_data_value = self.band.GetNoDataValue()
        arr = self.band.ReadAsArray(xoff=0, yoff=0,)
        arr = np.where(arr == no_data_value, np.nan, arr)
        return arr

def NDVI(band_4, band_8):
    molecule = band_8 - band_4
    denominator = band_8 + band_4
    return molecule / denominator

def array2raster(dst_rst_path, ref_rst_path, array, nodata_value=-9999):
    ref_rst_ds = gdal.Open(ref_rst_path)
    ref_band = ref_rst_ds.GetRasterBand(1)
    ref_rows = ref_rst_ds.RasterYSize
    ref_cols = ref_rst_ds.RasterXSize
    driver = ref_rst_ds.GetDriver()
    outRaster = driver.Create(dst_rst_path, ref_cols, ref_rows, 1, gdal.GDT_Float32)
    outRaster.SetGeoTransform(ref_rst_ds.GetGeoTransform())
    outband = outRaster.GetRasterBand(1)
    outband.SetNoDataValue(nodata_value)
    outband.WriteArray(array)
    # outRasterSRS = osr.SpatialReference()
    # outRasterSRS.ImportFromEPSG(4326)
    outRaster.SetProjection(ref_rst_ds.GetProjection())
    outband.FlushCache()
    outRaster = None

In [ ]:
RasterExtractor(satellite_imgs['20190327']['B04']).get_info()
plt.figure(figsize=(8, 8))
plt.imshow(RasterExtractor(satellite_imgs['20190327']['B04']).get_array())
plt.colorbar()

In [ ]:
RasterExtractor(satellite_imgs['20190327']['B08']).get_info()
plt.figure(figsize=(8, 8))
plt.imshow(RasterExtractor(satellite_imgs['20190327']['B08']).get_array())
plt.colorbar()

In [ ]:
# 用B04、B08資料算出NDVI
NDVI_arr = NDVI(RasterExtractor(satellite_imgs['20190327']['B04']).get_array(), 
                RasterExtractor(satellite_imgs['20190327']['B08']).get_array())

plt.figure(figsize=(8, 8))
plt.imshow(NDVI_arr)
plt.colorbar()

In [ ]:
# 把算出來的NDVI存成tiff檔
dst_path = os.path.join('output', 'ndvi_20190327.tif')
ref_path = os.path.join('data', '20190327', 'T50QRM_20190327T022551_B04.tif')

array2raster(dst_path, ref_path, NDVI_arr)